In [1]:
import pandas as pd
import cv2
import numpy as np 
from sklearn.model_selection import train_test_split

In [2]:
FILE_PATH = 'fer2013.csv'

In [3]:
image_size = (48, 48)
data = pd.read_csv(FILE_PATH)
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [4]:
pixels = data['pixels'].tolist()
width, height = image_size

In [5]:
# load images and emotions 
faces = []

for p in pixels:
    face = [int(pix) for pix in p.split(' ')]
    face = np.asarray(face).reshape(width, height)
    face = cv2.resize(face.astype('uint8'), image_size)
    faces.append(face.astype('float32'))

In [6]:

len(faces)

35887

In [7]:
faces = np.asarray(faces)
faces = np.expand_dims(faces, -1)

In [8]:

emotions = pd.get_dummies(data['emotion']).values
emotions.shape

(35887, 7)

In [9]:
def preprocess(x, v2=True):  # v2 to keep the image btw. -1 and 1
    x = x.astype('float32')
    x = x/255.0
    if v2:
        x = (x - 0.5)*2.0
    return x

In [10]:
faces = preprocess(faces)

In [11]:
xtrain, xtest, ytrain, ytest = train_test_split(faces, emotions,test_size=0.2,shuffle=True)

In [12]:
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras.layers import Activation, Convolution2D, Conv2D, Dropout, AveragePooling2D, BatchNormalization, GlobalAveragePooling2D, Flatten, Input, MaxPooling2D, SeparableConv2D

In [13]:
from keras.models import Model
from keras.regularizers import l2

In [14]:
batch_size = 32
epochs = 100
image_shape = (48, 48, 1)
verbose = True 
num_class = 7
patience = 50  # number of epochs with no improvement after which training will be stopped
base_path = 'models/'
l2_regularization = 0.01

In [15]:
data_generator = ImageDataGenerator(featurewise_center=False, featurewise_std_normalization=False, rotation_range=10, 
                                    width_shift_range=0.1, height_shift_range=0.1, zoom_range=.1, horizontal_flip=True)

In [16]:

regularization = l2(l2_regularization)

In [17]:
# model
image_input = Input(image_shape)
x = Conv2D(filters=8, kernel_size=(3,3), strides=(1,1), kernel_regularizer=regularization, use_bias=False)(image_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(filters=8, kernel_size=(3,3), strides=(1,1), kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# module 1
# residual module 
residual = Conv2D(filters=16, kernel_size=(1,1), strides=(2,2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(filters=16, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(filters=16, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)
x = layers.add([x,residual])

# module 2
# residual module 
residual = Conv2D(filters=32, kernel_size=(1,1), strides=(2,2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(filters=32, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(filters=32, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)
x = layers.add([x,residual])

# module 3
# residual module 
residual = Conv2D(filters=64, kernel_size=(1,1), strides=(2,2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(filters=64, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(filters=64, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)
x = layers.add([x,residual])

# module 4
# residual module 
residual = Conv2D(filters=128, kernel_size=(1,1), strides=(2,2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(filters=128, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(filters=128, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)
x = layers.add([x,residual])

x = Conv2D(filters=num_class, kernel_size=(3,3), padding='same')(x)
x = GlobalAveragePooling2D()(x)

output = Activation('softmax', name='predictions')(x)

In [29]:
model = Model(image_input, output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 48, 48, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 46, 46, 8)    72          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 46, 46, 8)    32          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 46, 46, 8)    0           batch_normalization[0][0]        
____________________________________________________________________________________________

In [ ]:
model.fit(xtrain, ytrain, validation_data=(xtest, ytest), epochs=100, batch_size = 32)

Epoch 1/100
898/898 [==============================] - 174s 192ms/step - loss: 1.8078 - accuracy: 0.3300 - val_loss: 1.5379 - val_accuracy: 0.4416
Epoch 2/100
898/898 [==============================] - 174s 193ms/step - loss: 1.4227 - accuracy: 0.4705 - val_loss: 1.3969 - val_accuracy: 0.4911
Epoch 3/100
898/898 [==============================] - 173s 192ms/step - loss: 1.3007 - accuracy: 0.5196 - val_loss: 1.4443 - val_accuracy: 0.4726
Epoch 4/100
898/898 [==============================] - 175s 195ms/step - loss: 1.2078 - accuracy: 0.5533 - val_loss: 1.3692 - val_accuracy: 0.5085
Epoch 5/100
898/898 [==============================] - 173s 192ms/step - loss: 1.1493 - accuracy: 0.5747 - val_loss: 1.2588 - val_accuracy: 0.5425
Epoch 6/100
898/898 [==============================] - 173s 193ms/step - loss: 1.0955 - accuracy: 0.6014 - val_loss: 1.1894 - val_accuracy: 0.5722
Epoch 7/100
898/898 [==============================] - 172s 192ms/step - loss: 1.0477 - accuracy: 0.6159 - val_loss: 1

In [ ]:
model_json = model.to_json()
with open("emodel.json", "w") as json_file:
   json_file.write(model_json)
model.save_weights("emodel.h5")
print("Saved model to disk")

In [ ]:
import os
import cv2
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image

In [ ]:
model_json = model.to_json()
model = model_from_json(open("emodel.json", "r").read())
model.load_weights('emodel.h5')
face_haar_cascade = cv2.CascadeClassifier(r'F:\ronil be project\project\facial emotion\haarcascade_frontalface_default.xml')
print(5)
model.save("emodel.h5")

In [ ]:
from keras.models import load_model
from time import sleep
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np
import os
print(5)
classifier =load_model('emodel.h5')

In [ ]:
class_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
cap=cv2.VideoCapture(0)

while True:
    ret, image = cap.read()

    converted_image= cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


    faces_detected = face_haar_cascade.detectMultiScale(converted_image)
    for (x,y,w,h) in faces_detected:
        cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = converted_image[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)
    # rect,face,image = face_detector(frame)


        if np.sum([roi_gray])!=0:
            roi = roi_gray.astype('float')/255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)

        # make a prediction on the ROI, then lookup the class

            preds = classifier.predict(roi)[0]
            label=class_labels[preds.argmax()]
            label_position = (x,y)
            cv2.putText(image,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
        else:
            cv2.putText(image,'No Face Found',(20,60),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
    cv2.imshow('Emotion Detector',image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
cap.release()
cv2.destroyAllWindows